# Imports

In [1]:
import os
import sys
import copy
#from pypcd.pypcd import make_xyz_rgb_point_cloud
import numpy as np
from tqdm import tqdm
import tensorflow as tf

import cv2
import pcl
import matplotlib.pyplot as plt

from utils import tfrecord_utils

# Constants

In [2]:
dataset_dirpath_in = '/media/daniel/monster-data/rgbd-dataset-raw'
dataset_dirpath_out = '/media/daniel/monster-data/rgbd-dataset-tfrecords'
test_instance_ids_filepath = 'dataset/testinstance_ids.txt'
max_point_cloud_size = 44214

# Dataset helper functions

In [3]:
def dataset_load_filepaths(dataset_path, use_depth=False, use_pcd=False):
    """
    Create datatset with filepaths. You can use either depth or pcd files.
    """
    if (use_depth and use_pcd) or (not use_depth and not use_pcd):
        assert ValueError, 'which files do you want to use?'
    
    dataset = {}
    classes = [f for f in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, f))]
    classes.sort()
    for cls in tqdm(classes):
        dataset[cls] = {}
        cls_path = os.path.join(dataset_path, cls)
        instances = os.listdir(cls_path)
        instances.sort(key=lambda x: int(x.split('_')[-1]))
        for inst in instances:
            inst_path = os.path.join(cls_path, inst)
            dataset[cls][inst] = {'img' : [], 'pcd' : [], 'loc': []}
            inst_files = os.listdir(inst_path)
            # Number of files
            pcd_filenames = [f for f in inst_files if '.pcd' in f]
            img_filenames = [f for f in inst_files if '_crop.png' in f]
            loc_filenames = [f for f in inst_files if '_loc.txt' in f]
            dpt_filenames = [f for f in inst_files if '_depthcrop.png' in f]
            # Get number of sequences
            if use_depth:
                key_filenames = dpt_filenames
                key_filenames.sort(key=lambda x: int(x.split('_')[-3]))
                seq_number = int(key_filenames[-1].split('_')[-3])
            if use_pcd:
                key_filenames = pcd_filenames
                key_filenames.sort(key=lambda x: int(x.split('_')[2]))
                seq_number = int(key_filenames[-1].split('_')[2])
            # For every sequence
            for seq in range(1, seq_number+1):
                inst_seq_name = inst + '_' + str(seq) + '_'
                inst_seq_key_filenames = [f for f in key_filenames if inst_seq_name in f]
                if use_depth:
                    inst_seq_key_filenames.sort(key=lambda x:int(x.split('_')[-2]))
                if use_pcd:
                    inst_seq_key_filenames.sort(key=lambda x:int(x.split('_')[-1].split('.')[0]))
                inst_seq_key_filenames = inst_seq_key_filenames[0::5]
                # For each file 
                for key_filename in inst_seq_key_filenames:
                    if use_depth:
                        basename = '_'.join(key_filename.split('_')[:-1])
                    if use_pcd:  
                        basename = key_filename.split('.')[0]
                    img_filename = [f for f in img_filenames if basename + '_crop.png' in f]
                    if len(img_filename) != 1:
                        print('problem with class: {} instance: {} basename: {}'.format(
                            cls, inst, basename))
                    loc_filename = [f for f in loc_filenames if basename + '_loc.txt' in f]
                    if len(loc_filename) != 1:
                        print('problem with class: {} instance: {} basename: {}'.format(
                            cls, inst, basename))
                    img_filepath = os.path.join(inst_path, img_filename[0])
                    loc_filepath = os.path.join(inst_path, loc_filename[0])
                    key_filepath = os.path.join(inst_path, key_filename)
                    # Remember
                    dataset[cls][inst]['img'].append(img_filepath)
                    dataset[cls][inst]['loc'].append(loc_filepath)
                    dataset[cls][inst]['pcd'].append(key_filepath)
    return dataset

def load_cross_validation_split(test_instance_ids_filepath):
    cross_test_split = []
    with open(test_instance_ids_filepath, 'r') as file:
        data = file.readlines()
    trials = np.sum(['******' in line for line in data])
    for trial in range(1, trials+1):
        test_inst_trial = []
        start_line_idx = [idx for idx, line in enumerate(data) if '****** trial ' + str(trial) in line][0]
        end_line_idx = [idx for idx, line in enumerate(data) if '****** trial ' + str(trial +1) in line]
        if len(end_line_idx) == 0:
            end_line_idx = len(data)-1
        else:
            end_line_idx = end_line_idx[0]
        for line_idx in range(start_line_idx+1, end_line_idx):
            test_inst = data[line_idx].strip()
            if len(test_inst):
                test_inst_trial.append(test_inst)
        cross_test_split.append(test_inst_trial)
    return cross_test_split

def cross_validation_split(dataset_paths, test_split):
    dataset_train = copy.deepcopy(dataset_paths)
    dataset_test = {}
    for test_instance in test_split:
        class_name = '_'.join(test_instance.split('_')[:-1])
        if class_name not in dataset_test:
            dataset_test[class_name] = {}
        dataset_test[class_name][test_instance] = dataset_train[class_name].pop(test_instance)
    return dataset_train, dataset_test

def dataset_make_flat(dataset): 
    flat_dataset = copy.deepcopy(dataset)
    for class_name in dataset:
        flat_dataset[class_name] = {'img' : [], 'pcd': [], 'loc': []}
        for key in ['img', 'pcd', 'loc']:
            for instance_name in dataset[class_name]:
                flat_dataset[class_name][key] = np.concatenate(
                    (flat_dataset[class_name][key], dataset[class_name][instance_name][key]))
    return flat_dataset

def dataset_convert_to_x_and_y(dataset, class_names, shuffle=True):
    X = {'img' : [], 'pcd': [], 'loc': []}
    Y = {'name': [], 'int': []}
    for class_idx, class_name in enumerate(class_names):
        for key in ['img', 'pcd', 'loc']:
            x_key = dataset[class_name][key]
            X[key] = np.concatenate((X[key], x_key))
        y_name = np.array([class_name]*len(x_key))
        y_int = np.array([class_idx]*len(x_key))
        Y['int'] = np.concatenate((Y['int'], y_int)).astype(int)
        Y['name'] = np.concatenate((Y['name'], y_name))
    if shuffle:
        indices = np.arange(len(Y['name']))
        np.random.shuffle(indices)
        for key in ['img', 'pcd', 'loc']:
            X[key] = X[key][indices]
        Y['int'] = Y['int'][indices]
        Y['name'] = Y['name'][indices]
    return X, Y

def load_pcd(filepath):
    pcd = pcl.load(filepath)
    return np.asarray(pcd)

def tfrecord_generate_data(X, Y, output_dir, split, pad_for='pointnet'):
    global max_point_cloud_size
    # tfrecords count
    num_per_shard = 1500
    num_shards = int(np.ceil(len(Y['name'])/num_per_shard))
    
    # Start TF session
    with tf.Graph().as_default():
        with tf.Session('') as _:
            for shard_id in range(num_shards):
                shard_filename = '%s_%s_%05d-of-%05d.tfrecord' % (split, 'rgbd', shard_id, num_shards)
                shard_filepath = os.path.join(output_dir, shard_filename)
                with tf.python_io.TFRecordWriter(shard_filepath) as tfrecord_writer:
                    start_idx = shard_id * num_per_shard
                    end_idx = min((shard_id + 1) * num_per_shard, len(Y['name']))
                    for data_idx in tqdm(range(start_idx, end_idx)):
                        # Load PCD/PNG
                        x_cloud = load_pcd(X['pcd'][data_idx])
                        y_int = Y['int'][data_idx]
                        y_name = str(Y['name'][data_idx])
                        if len(x_cloud) < max_point_cloud_size and pad_for=='pointnet':
                            pad_size = max_point_cloud_size - len(x_cloud)
                            x_cloud = np.pad(x_cloud, ((0, pad_size), (0, 0)), 'edge')
                        tf_example = tfrecord_utils.point_cloud_to_tfexample(x_cloud, y_name, y_int)
                        tfrecord_writer.write(tf_example.SerializeToString())
                        
def tfrecord_generate_paths(X, Y, output_dir, split,):

    # tfrecords count
    num_per_shard = 10**5
    num_shards = int(np.ceil(len(Y['name'])/num_per_shard))
    
    # Start TF session
    with tf.Graph().as_default():
        with tf.Session('') as _:
            for shard_id in range(num_shards):
                shard_filename = '%s_%s_%05d-of-%05d.tfrecord' % (split, 'rgbd', shard_id, num_shards)
                shard_filepath = os.path.join(output_dir, shard_filename)
                with tf.python_io.TFRecordWriter(shard_filepath) as tfrecord_writer:
                    start_idx = shard_id * num_per_shard
                    end_idx = min((shard_id + 1) * num_per_shard, len(Y['name']))
                    for data_idx in tqdm(range(start_idx, end_idx)):
                        x_img_path = X['img'][data_idx]
                        x_pcd_path = X['pcd'][data_idx]
                        x_loc_path = X['loc'][data_idx]
                        y_int = Y['int'][data_idx]
                        y_name = str(Y['name'][data_idx])
                        tf_example = tfrecord_utils.paths_to_tfexample(x_pcd_path, x_img_path, x_loc_path, y_name, y_int)
                        tfrecord_writer.write(tf_example.SerializeToString())

# Load dataset

In [4]:
use_depth_image=True
use_pcd_files=False
dataset_paths = dataset_load_filepaths(dataset_dirpath_in, use_depth_image, use_pcd_files)
cross_test_split = load_cross_validation_split(test_instance_ids_filepath)

100%|██████████| 51/51 [00:17<00:00,  2.84it/s]


# For each split

In [9]:
if not os.path.exists(dataset_dirpath_out):
    os.makedirs(dataset_dirpath_out)
    
for split_idx, test_split in enumerate(cross_test_split[:1]):
    ################################################################
    # Make dir
    ################################################################
    
    split_dirpath = os.path.join(dataset_dirpath_out, str(split_idx+1))
    if not os.path.exists(split_dirpath):
        os.makedirs(split_dirpath)
    
    ################################################################
    # Split dataset
    ################################################################
    
    train_filepaths, test_filepaths = cross_validation_split(dataset_paths, test_split)
    train_filepaths = dataset_make_flat(train_filepaths)
    test_filepaths = dataset_make_flat(test_filepaths)
    class_names = list(train_filepaths)
    class_names.sort()
    
    ################################################################
    # Prepare train data
    ################################################################
    
    train_dirpath = os.path.join(split_dirpath, 'train')
    if not os.path.exists(train_dirpath):
        os.makedirs(train_dirpath)
    
    depth_min = np.inf
    depth_max = -np.inf
    depths = []
    
    X, Y = dataset_convert_to_x_and_y(train_filepaths, class_names, shuffle=True)
    for depth_filepath in tqdm(X['pcd']):
        depth = cv2.imread(depth_filepath, cv2.IMREAD_ANYDEPTH)
        depth = depth.astype(np.float32)
        depths.append(depth.reshape(-1))
#         if depth_min > np.min(depth):
#             depth_min = np.min(depth)
#         if depth_max < np.max(depth):
#             depth_max = np.max(depth)

    depths = np.concatenate(depths)
    print('depth:', depths.shape)
    asdasd
    tfrecord_generate_paths(X, Y, train_dirpath, 'train')
    
    ################################################################
    # Prepare test data
    ################################################################
    
    test_dirpath = os.path.join(split_dirpath, 'test')
    if not os.path.exists(test_dirpath):
        os.makedirs(test_dirpath)
        
    X, Y = dataset_convert_to_x_and_y(test_filepaths, class_names, shuffle=False)
    tfrecord_generate_paths(X, Y, test_dirpath, 'test')

100%|██████████| 34898/34898 [23:27<00:00, 24.79it/s]


depth: (457510099,)


NameError: name 'asdasd' is not defined

In [10]:
np.mean(depths), np.std(depths)

(775.6092, 499.1676)

# Convert depth image to point cloud

In [ ]:
pcd_filepath = '/media/daniel/Atos/rgbd-dataset/banana/banana_1/banana_1_1_1.pcd'
depth_filepath = '/media/daniel/Atos/rgbd-dataset/banana/banana_1/banana_1_1_1_depthcrop.png'
image_filepath = '/media/daniel/Atos/rgbd-dataset/banana/banana_1/banana_1_1_1_crop.png'
crop_top_left_corner = [227, 245]

color = cv2.imread(image_filepath)
depth = cv2.imread(depth_filepath, cv2.IMREAD_ANYDEPTH)
plt.imshow(depth)
plt.show()

print(depth.shape, depth.dtype, depth[0][0])
print(np.max(depth))

In [ ]:
# Depth conversion
depth = depth.astype(np.float32)
depth[depth == 0] = np.nan
print(depth.shape, depth.dtype, depth[0][0])
print(np.max(depth))

In [ ]:
def create_point_cloud(depth_image, crop_top_left_corner=None, color_image=None):
    # Depth conversion
    depth = depth_image.astype(np.float32)
    depth[depth == 0] = np.nan
    # RGB-D camera constants
    center = [320, 240]
    image_h, image_w = depth.shape
    scale_f = 570.3
    mm_per_m = 1000
    if crop_top_left_corner is None:
        crop_top_left_corner = [0, 0]
    # Convert depth image to 3d point cloud
    channels = 3
    if color_image is not None:
        channels = 6
    point_cloud = np.zeros((image_h, image_w, channels), dtype=np.float32)
    x_grid = np.ones((image_h, 1), dtype=np.float32) * np.arange(image_w) + crop_top_left_corner[0] - center[0]
    y_grid = (np.arange(image_h).reshape(image_h, 1)*np.ones((1, image_w), dtype=np.float32) +
              crop_top_left_corner[1] - center[1])
    point_cloud[..., 0] = np.multiply(x_grid, depth) / scale_f / mm_per_m
    point_cloud[..., 1] = np.multiply(y_grid, depth) / scale_f / mm_per_m
    point_cloud[..., 2] = depth / mm_per_m
    # Assign color to point cloud
    if color_image is not None:
        point_cloud[..., 3:] = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB).astype(np.float32) / 255
    # Cut off to far points
    point_cloud[point_cloud[..., 2] > 2.5] = np.nan
    return point_cloud


point_cloud = create_point_cloud(depth, crop_top_left_corner)
temp = pcl.PointCloud(point_cloud.reshape(-1, 3))
pcl.save(temp, '/home/daniel/temp.pcd')